In [18]:
# imports 
import numpy as np
from math import sqrt 
from pprint import pprint
from numpy import array
from scipy.io import arff
import pandas as pd

In [19]:
# load data
df=pd.read_csv('ALOI_norm.csv')  

del df['id']
del df['Unnamed: 0']
df['outlier'] = df.outlier.apply(lambda label: 1 if label == "'yes'" else 0)

df.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,...,att19,att20,att21,att22,att23,att24,att25,att26,att27,outlier
0,0.784999,0.000018,0.0,0.000093,0.0,0.0,0.0,0.0,0.0,0.139811,...,0.048171,0.001189,0.0,0.115728,0.023449,0.0002,0.000220,0.034952,0.046914,1
1,0.958088,0.000000,0.0,0.001671,0.0,0.0,0.0,0.0,0.0,0.019556,...,0.000000,0.000000,0.0,0.022035,0.007516,0.0000,0.000000,0.001033,0.000000,1
2,0.938768,0.000000,0.0,0.005146,0.0,0.0,0.0,0.0,0.0,0.018451,...,0.000000,0.000000,0.0,0.035542,0.011982,0.0000,0.001595,0.019520,0.000000,1
3,0.954775,0.000000,0.0,0.001427,0.0,0.0,0.0,0.0,0.0,0.024944,...,0.000000,0.000000,0.0,0.019941,0.000805,0.0000,0.000000,0.000035,0.000000,1
4,0.933601,0.000000,0.0,0.001682,0.0,0.0,0.0,0.0,0.0,0.037002,...,0.000000,0.000000,0.0,0.046759,0.002663,0.0000,0.000339,0.001359,0.000000,1


In [20]:
df.shape

(50000, 28)

In [21]:
# data to values
df = df.values

In [22]:
# Extratrees Deep
import argparse
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/aloi.json")
gc = GCForest(config)

In [23]:
# train test 
from sklearn.cross_validation import train_test_split
y = df[:,27]
X = df[:,0:27]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [24]:
# number of class
len(np.unique(y_train))

2

In [26]:
         # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
   

[ 2018-04-25 00:21:24,562][cascade_classifier.fit_transform] X_groups_train.shape=[(33500, 27)],y_train.shape=(33500,),X_groups_test.shape=[(16500, 27)],y_test.shape=(16500,)
[ 2018-04-25 00:21:24,568][cascade_classifier.fit_transform] group_dims=[27]
[ 2018-04-25 00:21:24,570][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 00:21:24,571][cascade_classifier.fit_transform] group_ends=[27]
[ 2018-04-25 00:21:24,573][cascade_classifier.fit_transform] X_train.shape=(33500, 27),X_test.shape=(16500, 27)
[ 2018-04-25 00:21:24,578][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(33500, 27), X_cur_test.shape=(16500, 27)
[ 2018-04-25 00:21:25,504][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.49%
[ 2018-04-25 00:21:26,370][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.34%
[ 2018-04-25 00:21:27,222][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estim

In [27]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-25 00:21:33,536][cascade_classifier.transform] X_groups_test.shape=[(16500, 27)]
[ 2018-04-25 00:21:33,539][cascade_classifier.transform] group_dims=[27]
[ 2018-04-25 00:21:33,540][cascade_classifier.transform] X_test.shape=(16500, 27)
[ 2018-04-25 00:21:33,543][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(16500, 27)


Test Accuracy of ExtraTrees = 97.042424 %


In [28]:
# Make data + estimators
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-25 00:21:35,076][cascade_classifier.transform] X_groups_test.shape=[(16500, 27)]
[ 2018-04-25 00:21:35,079][cascade_classifier.transform] group_dims=[27]
[ 2018-04-25 00:21:35,080][cascade_classifier.transform] X_test.shape=(16500, 27)
[ 2018-04-25 00:21:35,082][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(16500, 27)


In [29]:
# New data
X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-25 00:21:36,457][cascade_classifier.fit_transform] X_groups_train.shape=[(33500, 29)],y_train.shape=(33500,),X_groups_test.shape=[(16500, 29)],y_test.shape=(16500,)
[ 2018-04-25 00:21:36,461][cascade_classifier.fit_transform] group_dims=[29]
[ 2018-04-25 00:21:36,462][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 00:21:36,464][cascade_classifier.fit_transform] group_ends=[29]
[ 2018-04-25 00:21:36,465][cascade_classifier.fit_transform] X_train.shape=(33500, 29),X_test.shape=(16500, 29)
[ 2018-04-25 00:21:36,469][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(33500, 29), X_cur_test.shape=(16500, 29)
[ 2018-04-25 00:21:37,204][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.85%
[ 2018-04-25 00:21:38,094][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.82%
[ 2018-04-25 00:21:38,963][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estim

In [30]:
# predict  data + Es
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-25 00:21:45,411][cascade_classifier.transform] X_groups_test.shape=[(16500, 29)]
[ 2018-04-25 00:21:45,414][cascade_classifier.transform] group_dims=[29]
[ 2018-04-25 00:21:45,417][cascade_classifier.transform] X_test.shape=(16500, 29)
[ 2018-04-25 00:21:45,420][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(16500, 29)


Test Accuracy of ExtraTrees = 97.775758 %


In [31]:
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100),i,'fois')

In [32]:
# Deep
for i in range (20):
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    affiche()

[ 2018-04-25 00:21:46,849][cascade_classifier.fit_transform] X_groups_train.shape=[(33500, 29)],y_train.shape=(33500,),X_groups_test.shape=[(16500, 29)],y_test.shape=(16500,)
[ 2018-04-25 00:21:46,852][cascade_classifier.fit_transform] group_dims=[29]
[ 2018-04-25 00:21:46,854][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 00:21:46,856][cascade_classifier.fit_transform] group_ends=[29]
[ 2018-04-25 00:21:46,857][cascade_classifier.fit_transform] X_train.shape=(33500, 29),X_test.shape=(16500, 29)
[ 2018-04-25 00:21:46,862][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(33500, 29), X_cur_test.shape=(16500, 29)
[ 2018-04-25 00:21:47,566][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.67%
[ 2018-04-25 00:21:48,550][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.70%
[ 2018-04-25 00:21:49,443][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estim

('Test Accuracy of ExtraTrees = 97.715152 %', 0, 'fois')


[ 2018-04-25 00:21:58,229][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.67%
[ 2018-04-25 00:21:59,102][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=98.00%
[ 2018-04-25 00:21:59,969][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.64%
[ 2018-04-25 00:22:00,787][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.73%
[ 2018-04-25 00:22:01,621][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=97.73%
[ 2018-04-25 00:22:02,485][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.58%
[ 2018-04-25 00:22:03,419][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.79%
[ 2018-04-25 00:22:04,422][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=97.79%


('Test Accuracy of ExtraTrees = 97.606061 %', 1, 'fois')


[ 2018-04-25 00:22:08,773][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.58%
[ 2018-04-25 00:22:09,692][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.76%
[ 2018-04-25 00:22:10,604][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.76%
[ 2018-04-25 00:22:11,487][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.82%
[ 2018-04-25 00:22:12,349][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=97.82%
[ 2018-04-25 00:22:13,200][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.46%
[ 2018-04-25 00:22:14,097][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.67%
[ 2018-04-25 00:22:14,972][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=97.97%


...

('Test Accuracy of ExtraTrees = 97.230303 %', 17, 'fois')


[ 2018-04-25 00:24:56,852][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.43%
[ 2018-04-25 00:24:57,753][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.46%
[ 2018-04-25 00:24:58,616][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.34%
[ 2018-04-25 00:24:59,455][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.61%
[ 2018-04-25 00:25:00,322][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=97.58%
[ 2018-04-25 00:25:01,334][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.34%
[ 2018-04-25 00:25:02,223][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.34%
[ 2018-04-25 00:25:03,074][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=97.61%


('Test Accuracy of ExtraTrees = 97.260606 %', 18, 'fois')


[ 2018-04-25 00:25:07,094][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.34%
[ 2018-04-25 00:25:08,062][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.34%
[ 2018-04-25 00:25:08,917][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.37%
[ 2018-04-25 00:25:09,719][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.37%
[ 2018-04-25 00:25:10,592][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=97.46%
[ 2018-04-25 00:25:11,466][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.52%
[ 2018-04-25 00:25:12,451][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.28%
[ 2018-04-25 00:25:13,289][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=97.58%


('Test Accuracy of ExtraTrees = 97.206061 %', 19, 'fois')


# ('Test Accuracy of ExtraTrees = 97.715152 %', 0, 'fois')